**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [9]:
# Get a list of available tables 
tables = list(client.list_tables(dataset)) # Your code here

# Print your answer
list_tables = [table.table_id for table in tables]
print(list_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> Incorrect value for `list_of_tables`: `[<google.cloud.bigquery.table.TableListItem object at 0x7fe560d06f28>, <google.cloud.bigquery.table.TableListItem object at 0x7fe5623626d8>, <google.cloud.bigquery.table.TableListItem object at 0x7fe5623626a0>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d0b390>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d0b2e8>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d0b6d8>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d0b6a0>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d0b5c0>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d0b320>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d0b3c8>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d255c0>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d25358>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560d250b8>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560cde390>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560cde2b0>, <google.cloud.bigquery.table.TableListItem object at 0x7fe560cde358>]`

For the solution, uncomment the line below.

In [6]:
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] 

```

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [10]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,55832858,None,<p>There may be privacy description <em>(for S...,None,None,0,None,2019-04-24 14:45:59.090000+00:00,None,2019-04-24 14:45:59.090000+00:00,None,None,None,None,7266317,6004032,2,0,None,None
1,55832872,None,<p>How do I set the name of the button to the ...,None,None,3,None,2019-04-24 14:46:52.690000+00:00,None,2019-04-24 14:46:52.690000+00:00,None,None,None,None,5873109,55832746,2,0,None,None
2,55832876,None,<pre><code>go test -v -timeout 30s &lt;path_to...,None,None,0,None,2019-04-24 14:47:09.227000+00:00,None,2019-04-24 14:47:09.227000+00:00,None,None,None,None,5713047,16935965,2,0,None,None
3,55832878,None,<p>I suggest you create one custom pin represe...,None,None,0,None,2019-04-24 14:47:10.297000+00:00,None,2019-04-24 14:47:10.297000+00:00,None,None,None,None,6001090,33622927,2,0,None,None
4,55832879,None,"<p>You have to use <a href=""https://en.cpprefe...",None,None,2,None,2019-04-24 14:47:16.853000+00:00,None,2019-04-24 14:47:16.853000+00:00,None,None,None,None,10765031,55831782,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [11]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,56412356,S3 Multipart upload with pause and resume func...,<p>I am trying to acheive s3 multipart upload ...,None,0,0,None,2019-06-02 05:09:24.723000+00:00,None,2019-06-02 05:09:24.723000+00:00,None,None,None,None,7225816,None,1,0,<javascript><amazon-s3><stream>,1
1,56318079,Angular Nginx Docker 404,<p>Been driving myself nuts trying to figure t...,None,1,0,None,2019-05-26 23:14:43.403000+00:00,None,2019-05-26 23:21:11.503000+00:00,None,None,None,None,1058951,None,1,0,<angular><docker><nginx><http-status-code-404>,257
2,56136152,Switch structure control for OSC msg,<p>I’m new to SC and the whole music programmi...,None,0,0,None,2019-05-14 18:05:32.077000+00:00,None,2019-05-14 18:05:32.077000+00:00,None,None,None,None,8880735,None,1,0,<supercollider>,2
3,56207713,AWS QuickSight - Smooth line graph/plot,<p>do you know if ''''AWS QuickSight'''' has a...,None,0,0,None,2019-05-19 12:15:57.697000+00:00,None,2019-05-19 12:15:57.697000+00:00,None,None,None,None,11036699,None,1,0,<amazon>,2
4,56250259,I am not able to access kubernetes dash-board ...,<p>I have installed docker and kubernetes in m...,None,0,0,None,2019-05-22 06:03:36.240000+00:00,None,2019-05-22 06:03:36.240000+00:00,None,None,None,None,9493974,None,1,0,<kubernetes-dashboard>,2


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [12]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [15]:
# Your code here
questions_query = """
                  SELECT ID, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         ID                                              title  owner_user_id
0  56217818  BigQuery: How do DML operations affect storage...       189336.0
1  56187040  Unable to setup custom schedules in Bigquery D...      4749660.0
2  56223596  Is it possible to UNNEST an array in BigQuery ...      2696815.0
3  56240285  Google Big Query Storage API Inconsistently Re...      1113712.0
4  56264424        FLATTEN results using MAX value in BigQuery       260826.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [24]:
# Your code here
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                        ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

BadRequest: 400 GET https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/queries/fe41e2ba-4f52-4d6f-b0bd-6642f8b5c5bc?maxResults=0&location=US: Query exceeded limit for bytes billed: 10000000000. 22801285120 or higher required.

For a hint or the solution, uncomment the appropriate line below.

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [30]:
# Your code here
bigquery_experts_query = """
                SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                        ON q.id = a.parent_Id
                WHERE q.tags LIKE '%bigquery%'
                GROUP BY a.owner_user_id
                """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config = safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

     user_id  number_of_answers
0  7813005.0                 25
1   454137.0                  2
2  9908251.0                  9
3  5691525.0                 10
4  3058302.0                  8


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [31]:
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%{topic}%'
               GROUP BY a.owner_user_id
               """
               
    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)
    
    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results

```

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=sizeAll&filetype=fileTypeBigQuery).

# Feedback

Bring any questions or feedback to the [Learn Discussion Forum](https://www.kaggle.com/learn-forum).

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*